# Seq2seq Learning - Keras

- Unequal lengths
- Equal lengths

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print('TensorFlow Version: {}'.format(tf.__version__))

Found GPU at: /device:GPU:0
TensorFlow Version: 1.12.0


In [0]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, RepeatVector
from keras.layers import Conv1D, GlobalMaxPool1D, Flatten, Bidirectional, MaxPooling1D
from keras.layers import Activation, TimeDistributed, Dropout, Embedding
from keras import regularizers
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Model, model_from_json
from keras.layers import Input, Dense
from keras.utils import to_categorical

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


## Downloading dataset

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

## Unequal lengths

In [0]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'gdrive/My Drive/Colab Notebooks/data/fra.txt'

In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 16
Max sequence length for outputs: 59


### Char by char

In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=10, #epochs,
          validation_split=0.2)

# Save model
# model.save_weights('models/s2s.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_35 (InputLayer)           (None, None, 71)     0                                            
__________________________________________________________________________________________________
input_36 (InputLayer)           (None, None, 94)     0                                            
__________________________________________________________________________________________________
lstm_34 (LSTM)                  [(None, 256), (None, 335872      input_35[0][0]                   
__________________________________________________________________________________________________
lstm_35 (LSTM)                  [(None, None, 256),  359424      input_36[0][0]                   
                                                                 lstm_34[0][1]                    
          

In [0]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    print('Target sentence:', target_texts[seq_index])

-
Input sentence: Go.
Decoded sentence: Allez vous prêtes !

Target sentence: 	Va !

-
Input sentence: Run!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Cours !

-
Input sentence: Run!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Courez !

-
Input sentence: Fire!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Au feu !

-
Input sentence: Help!
Decoded sentence: Arrête de la mais.

Target sentence: 	À l'aide !

-
Input sentence: Jump.
Decoded sentence: Viens partir.

Target sentence: 	Saute.

-
Input sentence: Stop!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Stop !

-
Input sentence: Stop!
Decoded sentence: Arrête de la mainten !

Target sentence: 	Arrête-toi !

-
Input sentence: Wait!
Decoded sentence: Attendez à l'ait !

Target sentence: 	Attends !



### Word by word

- find set of words
- convert words to indices
- create dataset

In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
x, _, _ = decoder_lstm(x, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
print(model.summary())

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 256)    18176       input_31[0][0]                   
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 256)    24064       input_32[0][0]                   
__________________________________________________________________________________________________
lstm_30 (L

ValueError: Error when checking input: expected input_31 to have 2 dimensions, but got array with shape (10000, 16, 71)

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
docs = ['Well done!',
        'Good work',
        'Great effort',
        'nice work',
        'Excellent!',
        'Weak',
        'Poor effort!',
        'not good',
        'poor work',
        'Could have done better']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[27, 8], [38, 20], [48, 29], [46, 20], [23], [24], [43, 29], [23, 38], [43, 20], [47, 7, 8, 20]]
[[27  8  0  0]
 [38 20  0  0]
 [48 29  0  0]
 [46 20  0  0]
 [23  0  0  0]
 [24  0  0  0]
 [43 29  0  0]
 [23 38  0  0]
 [43 20  0  0]
 [47  7  8 20]]


In [0]:
# define the model
inputs = Input(shape=(max_length,))
x = Embedding(vocab_size, 8)(inputs)
x = Flatten()(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)
print(model.summary())

# fit the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(padded_docs, labels, epochs=200, verbose=0)

# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_45 (InputLayer)        (None, 4)                 0         
_________________________________________________________________
embedding_18 (Embedding)     (None, 4, 8)              400       
_________________________________________________________________
flatten_8 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None
Accuracy: 100.000000


In [0]:
preds = model.predict(padded_docs)
preds

array([[0.7861039 ],
       [0.72447515],
       [0.69255024],
       [0.768694  ],
       [0.61293256],
       [0.34747598],
       [0.30596223],
       [0.38281503],
       [0.41705278],
       [0.09242197]], dtype=float32)

In [0]:
### another implementation

In [0]:
from random import randint
import numpy as np
from keras.utils import to_categorical
from keras.layers import Input, LSTM, Dense
from keras.models import Model

 
# generate a sequence of random integers
def generate_sequence(length, n_unique):
	return [randint(1, n_unique-1) for _ in range(length)]
 
# prepare data for the LSTM
def get_dataset(n_in, n_out, cardinality, n_samples):
	X1, X2, y = list(), list(), list()
	for _ in range(n_samples):
		# generate source sequence
		source = generate_sequence(n_in, cardinality)
		# define target sequence
		target = source[:n_out]
		target.reverse()
		# create padded input target sequence
		target_in = [0] + target[:-1]
		# encode
		src_encoded = to_categorical(source, num_classes=cardinality)
		tar_encoded = to_categorical(target, num_classes=cardinality)
		tar2_encoded = to_categorical(target_in, num_classes=cardinality)
		# store
		X1.append(src_encoded)
		X2.append(tar2_encoded)
		y.append(tar_encoded)
	return np.array(X1), np.array(X2), np.array(y)
 
# decode a one hot encoded string
def one_hot_decode(encoded_seq):
	return [np.argmax(vector) for vector in encoded_seq]
 
# configure problem
n_features = 50 + 1
n_steps_in = 6
n_steps_out = 3
# generate a single source and target sequence
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
print(X1.shape, X2.shape, y.shape)
print('X1=%s, X2=%s, y=%s' % (one_hot_decode(X1[0]), one_hot_decode(X2[0]), one_hot_decode(y[0])))

In [0]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    encoder_inputs = Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = Input(shape=(n_units,))
    decoder_state_input_c = Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model

# generate target given source sequence
def predict_sequence(infenc, infdec, source, n_steps, cardinality):
    # encode
    state = infenc.predict(source)
    # start of sequence input
    target_seq = np.array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
    # collect predictions
    output = list()
    for t in range(n_steps):
        # predict next char
        yhat, h, c = infdec.predict([target_seq] + state)
        # store prediction
        output.append(yhat[0,0,:])
        # update state
        state = [h, c]
        # update target sequence
        target_seq = yhat
    return np.array(output)

In [0]:
# generate training dataset
X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
print(X1.shape, X2.shape, y.shape)

In [0]:
# model
train, infenc, infdec = define_models(n_features, n_features, 128)
train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

# train model
train.fit([X1, X2], y, epochs=1)

In [0]:
# evaluate LSTM
total, correct = 100, 0
for _ in range(total):
	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
	if np.array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
		correct += 1
print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))

## Equal lengths

In [0]:
MAXLEN = 7
chars = '0123456789+ '
DIGITS = 3

RNN = LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

print('Build model...')
# model = Sequential()
# # "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# # Note: In a situation where your input sequences have a variable length,
# # use input_shape=(None, num_feature).
# model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# # As the decoder RNN's input, repeatedly provide with the last hidden state of
# # RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# # length of output, e.g., when DIGITS=3, max output is 999+999=1998.
# model.add(RepeatVector(DIGITS + 1))
# # The decoder RNN could be multiple layers stacked or a single layer.
# for _ in range(LAYERS):
#     # By setting return_sequences to True, return not only the last output but
#     # all the outputs so far in the form of (num_samples, timesteps,
#     # output_dim). This is necessary as TimeDistributed in the below expects
#     # the first dimension to be the timesteps.
#     model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# # Apply a dense layer to the every temporal slice of an input. For each of step
# # of the output sequence, decide which character should be chosen.
# model.add(TimeDistributed(Dense(len(chars))))
# model.add(Activation('softmax'))

inputs = Input(shape=(MAXLEN, len(chars)))
x = RNN(HIDDEN_SIZE)(inputs)
x = RepeatVector(DIGITS + 1)(x)
for _ in range(LAYERS):
    x = RNN(HIDDEN_SIZE, return_sequences=True)(x)
x = TimeDistributed(Dense(len(chars)))(x)
outputs = Activation('softmax')(x)
model = Model(inputs, outputs)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print(model.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_47 (InputLayer)        (None, 7, 12)             0         
_________________________________________________________________
lstm_42 (LSTM)               (None, 128)               72192     
_________________________________________________________________
repeat_vector_11 (RepeatVect (None, 4, 128)            0         
_________________________________________________________________
lstm_43 (LSTM)               (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_7 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_______________________________________________________

## Saving keras model

In [0]:
import json
from keras.models import model_from_json, load_model

# Option 1: Save Weights + Architecture
model.save_weights('model_weights.h5')
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())
# Option 1: Load Weights + Architecture
with open('model_architecture.json', 'r') as f:
    new_model_1 = model_from_json(f.read())
new_model_1.load_weights('model_weights.h5')

# Option 2: Save/load the entire model
model.save('my_model.h5')
new_model_2 = load_model('my_model.h5')

## Saving sklearn models

In [0]:
from sklearn.externals import joblib

# Saving a model
joblib.dump(my_model, '_model.pkl')

# Loading a model
loaded_model = joblib.load('my_model.pkl')